In [3]:
# EXERCISE 1a: Validate Diffie-Hellman parameters and generate secret shared key.

# Text to print if parameters are not valid:
#  - "invalid p"
#  - "invalid alpha"
#  - "invalid private keys"
#  - "invalid public keys"
# Text to print if all parameters are valid:
#  - "All correct"

# Return:
#  - if all parameters are correct: k_AB
#  - if any parameter is invalid: None

def UAB_check_Diffie_Hellman(p,alpha,a,b,k_pubA,k_pubB):
    
    #VALIDAR PARÀMETRES
    totValid = 0
    
    #P és primera?
    validP = is_prime(p)
    
    if not validP:
        print("invalid p")
    else:
        totValid += 1
    
    #Valors a i b entre 2, p-2
    if not (2 <= a <= p-2) or not (2 <= b <= p-2)  :
        print("invalid private keys")
    else:
        totValid += 1

    #Per saber si alpha és primitiu: alpha.multiplicative_order() = ϕ(n)
    #alpha.multiplicative = p - 1
    
    if alpha.multiplicative_order() == p - 1:
        totValid += 1
    else:
        print("invalid alpha")
    
    #Claus públiques alpha^priv mod p
    
    validPubA = power_mod(alpha, a, p)
    
    validPubB = power_mod(alpha, b, p)
    
    #Les claus calculades són iguals que les passades per paràmetre?
    if (validPubA != k_pubA) or (validPubB != k_pubB):
        print("invalid public keys")
    else:
        totValid += 1
        
    
    #GENERAR CLAU SECRETA
    #Kab = B^a mod p
    #Kba = A^b mod p
    #Kab = Kba
    
    Kab = power_mod(k_pubB, a, p)
    
    if totValid == 4:
        print("All correct")
        return Kab
    else:
        return None

    ######################################

# Tests

p = 149
Zp=GF(p)
alpha = 11
al=Zp(alpha)
a = 25
b = 63
k_pubA = 50
k_pubB = 87
print(UAB_check_Diffie_Hellman(p,al,a,b,k_pubA,k_pubB))

print("------------------")

alpha = 5
al=Zp(alpha)
a=200
print(UAB_check_Diffie_Hellman(p,al,a,b,k_pubA,k_pubB))

# Output

#All correct
#14
#invalid private keys
#invalid alpha
#invalid public keys
#None

All correct
14
------------------
invalid private keys
invalid alpha
invalid public keys
None


In [4]:
# EXERCISE 1b: Find the element 'a' such that alpha^a = k_pubA (mod p). 'a' must be searched using brute force.

# Return
# - discrete logarithm log_alpha(k_pubA), which is the private key 'a' derived from k_pubA in a Diffie-Hellman key exchange

def UAB_find_discrete_log(p,alpha,k_pubA):

    # Kpub = alpha^a mod p
    #Fem totes les possibles combinacions i si fent la fòrmula dona la clau pública
    #Clau privada serà correcte
    
    for a in range (p):
        if power_mod(alpha, a, p) == k_pubA:
            return a
    
    return None
    ######################################

# Test

p = 149
alpha = 11
k_pubA = 87

print(UAB_find_discrete_log(p,alpha,k_pubA))
# OUTPUT: 63

63


In [6]:
# EXERCISE 1c: Estimate the time (in years) it would take for your computer to find the private key corresponding
# to a public key using a 1024-bit prime number. 
# To perform this estimation, use smaller random values for the prime 'p' and the private key 'a'.
# In order to measure each execution time of UAB_find_discrete_log, use '%time' as follows:

# %time private_key=UAB_find_discrete_log(p,alpha,k_pubA)

# After each test, check that the private key returned by 'UAB_find_discrete_log' is the same as 'a'.
# In that case, print "Correct private key"

# Perform at least three tests for each number of bits, in order to obtain the average of different execution
# times, and fill the following table:

# Number of bits | avg. time
# 8 | 81 µs
# 12 | 2.88 ms
# 16 | 45.1 ms
# 20 | 705.7 ms
# 24 | 21.1 s

# Number of bits | three tests
# 8 | 107 µs, 16 µs, 120 µs
# 12 | 4.2 ms,3.83 ms, 606 µs
# 16 | 10.1 ms, 23.2 ms, 102 ms
# 20 | 542 ms,1.38 s, 195 ms
# 24 | 20.1 s, 13.6 s, 29.6 s

# Write your estimation here: 3.000.000.000 years aprox.

#PRIMER: TROBAR P ALEATÒRIA
#random prime (n, proof=None, lbound=2)
#n : superior (2** --> elevar a 8)
#lbound: inferior (elevat a 7)
bits = 12
p = random_prime (2**bits - 1, proof=None, lbound=2**(bits - 1))

#SEGON: TROBEM ALPHA ALEATÒRIA
alpha = primitive_root (p)

#TERCER: TROBEM CLAU PRIVADA
a = randint (2, p - 2) 

#QUART: CALCULAR K_PUBA
k_pubA = power_mod(alpha, a, p) 

#CALCULEM PRIV A PARTIR DE PÚBLICA

%time private_key=UAB_find_discrete_log(p,alpha,k_pubA)

if private_key == a:
    print("Correct private key")




########################################################


CPU times: user 3.1 ms, sys: 0 ns, total: 3.1 ms
Wall time: 3.1 ms
Correct private key


In [14]:
# EXERCISE 2: Implement the following functions to create and validate a table of all possible encrypted NIU+nota
# using RSA

# UAB_gen_RSA_NIU_nota_table: Generate a table containing all possible encrypted NIU+nota.
#   This table can be implemented as a Python dictionary
#   To avoid creating a huge table, we will assume that NIUs are in the range of 1600000-1700000
#   Returns a dictionary with all c->m values

def UAB_gen_RSA_NIU_nota_table(n,e,NIU_nota_min,NIU_nota_max):

    #### IMPLEMENTATION GOES HERE ####
        
    table = {}
    
    for m in range(NIU_nota_min, NIU_nota_max + 1):
        
        #Xifrar m --> c = m^e mod n
        c = power_mod(m, e, n) 
        table[c] = m  
    return table
    ######################################
    
# UAB_RSA_decrypt: decrypt c using private key d and return decrypted m.

def UAB_RSA_decrypt(n,c,d):

    #Desxifrar m = c^d mod n
    m = power_mod(c, d, n)

    return m
    
    ######################################
    


    
# UAB_RSA_table_tests: Create a table that shows the result of testing the NIU+nota values obtained
# with the above table.

# 1- generate public key (n,e) and private key 'd' from random primes p,q using 32 bits.
# 2- use UAB_gen_RSA_NIU_nota_table to create the c->NIU table
# 3- create a table with z entries with the following columns:
#   m | c | m_table | m_decrypt | test
#     m=random NIU+nota value in the range of NIU_nota_min - NIU_nota_max
#     m_table=NIU+nota obtained from the previous c->NIU table
#     m_decrypt=NIU+nota calculated using UAB_RSA_decrypt
#     test=result of validating that m == m_table == m_decrypt

# If your code is correct, test must be True for all z rows.
# Returns nothing.

def UAB_RSA_table_tests(z,NIU_nota_min,NIU_nota_max):

    bits=32

    #1 - generar clau pública (n, e) privada d amb random p, q 
    
    # P i Q
    p = random_prime (2**bits - 1, proof=None, lbound=2**(bits - 1))
    q = random_prime (2**bits - 1, proof=None, lbound=2**(bits - 1))

    
    # n = p*q
    n = p*q
    
    fiN = (p - 1)* (q - 1)
    
    #Triar enter e on mcd (e, fi(n)) = 1
    surt = False
    
    while surt == False:
        e = randint(2, fiN - 1)
        if gcd(e, fiN) == 1: 
            surt = True
        
    #Calcular d = e^-1 mod fiN
    d = inverse_mod (e, fiN)
    
    #2 - Dict
    dict_xifrat = UAB_gen_RSA_NIU_nota_table(n, e,NIU_nota_min, NIU_nota_max )
    
    print ("m | c | m_table | m_decrypt | test")

    for i in range(z):
        
        #Generar random m
        m = randint(NIU_nota_min, NIU_nota_max)
        
        #Xifrem
        c = power_mod(m, e, n)
        
        #Busquem el missatge xifrat dins del diccionari
        m_table = dict_xifrat.get(c, None)
        
        #Utilitzem funció anterior
        m_decrypt = UAB_RSA_decrypt(n,c,d)
        
        if m == m_table == m_decrypt:
            test = true
        else:
            test = false
        
        
        print(m," | ",c," | ",m_table," | ",m_decrypt," | ",test)    

    ######################################

    
# Test

NIU_nota_l=16000000
NIU_nota_h=17000009
z=5

UAB_RSA_table_tests(z,NIU_nota_l,NIU_nota_h)

# EXAMPLE OUTPUT:

#m | c | m_table | m_decrypt | test
#16909523  |  6581635045685005871  |  16909523  |  16909523  |  True
#16203719  |  8166046811892358677  |  16203719  |  16203719  |  True
#16475752  |  4564422918350688854  |  16475752  |  16475752  |  True
#16072580  |  255203817356383661  |  16072580  |  16072580  |  True
#16746914  |  7230994191997193625  |  16746914  |  16746914  |  True

m | c | m_table | m_decrypt | test
16963797  |  11238096658107566171  |  16963797  |  16963797  |  True
16212679  |  8177985816781453997  |  16212679  |  16212679  |  True
16429515  |  7295406761340970999  |  16429515  |  16429515  |  True
16116265  |  13265654311970806080  |  16116265  |  16116265  |  True
16607618  |  154277947153982944  |  16607618  |  16607618  |  True


In [11]:
# EXERCISE 3: Implement de following functions to create and validate the signatures created
# using the ElGamal existential forgery attack.

# UAB_ElGamal_verify: verifies that (r,s) is a valid signature of m using the public key (p,alpha,beta).
# returns True or False

def UAB_ElGamal_verify(r,s,p,alpha,beta,m):

    #Calcular t = B^r * r^s mod p
    Br = power_mod(beta, r, p)
    rs = power_mod(r, s, p)
    
    t = (Br * rs) % p
   
        
    t2 = power_mod(alpha, m, p)
    
    if (t == t2):
        return True
    else:
        return False

    
    ######################################
    


# UAB_ElGamal_sign_existential_forgery: using parameters i and j, returns the signature (r,s) and the message (m).
# Returns the tuple (r,s,m)
# If gcd(j,p-1) != 1 then returns None.

def UAB_ElGamal_sign_existential_forgery(p,alpha,beta,i,j):
    
    if gcd(j, p-1) != 1:
        return None
    
    #CALCULAR LA SIGNATURA
    
    #Calcular la r
    r1 = power_mod(alpha, i, p)
    r2 = power_mod(beta, j, p)
    r = (r1*r2) % p
    
    #Calcular la s
    s2 = inverse_mod(j,p-1)
    s = (-r*s2) % (p-1)
    
    #CALCULAR MISSATGE
    # m = si mod p - 1
    m = (s*i) % (p-1)

    return r,s,m

    ######################################
    
    

# UAB_ElGamal_sign_exist_forgery_test: Create a table with z entries with the following columns:
#  p | alpha | beta | r | s | | m | test
# 
#   p= random prime
#   alpha= primitive root modulo p
#   beta= alpha^d mod p (where d is the private key)
#   r, s, m = signature and message generated by the UAB_ElGamal_sign_existential_forgery function
#   test = result of UAB_ElGamal_verify, validating that (r,s) is a correct signature of m

# If your code is correct, test must be True for all z rows.
# Returns nothing.

def UAB_ElGamal_sign_exist_forgery_test(z):

    bits=16
    
    print ("p | alpha | beta | r | s | m | test")

    for i in range(z):
        
        # 1 - generar p, alpha,d ,beta
        
        bits = 16
        
        p = random_prime (2**bits - 1, proof=None, lbound=2**(bits - 1))
        
        alpha = randint(2, p-1)
        
        #Clau privada
        d = randint(2, p-2)
        
        beta = power_mod(alpha, d, p)
        
        # 2
        
        i = randint(2, p-1)
        j = randint(2, p-1)
        
        #Verificar que mcd (j, p-1) = 1
        surt = False
        while surt == False :
            j = randint(2, p-1)
            if gcd(j, p - 1) == 1:
                surt = True
            
    
            
        #Calcular r, s, m
        signatura = UAB_ElGamal_sign_existential_forgery(p, alpha, beta, i, j)
        
        #Si la signatura és None
        if signatura is None:
            continue  
        
        r, s, m = signatura  
        
        # 3 - signatura vàlida
        
        test = UAB_ElGamal_verify(r, s, p, alpha, beta, m)
        
        print (p ,"|", alpha ,"|", beta ,"|", r, "|", s, "|", m ,"|" ,test)

    ######################################

    return None
    
# Test

z=10

UAB_ElGamal_sign_exist_forgery_test(z)

# EXAMPLE OUTPUT

# p | alpha | beta | r | s | | m | check
# 58631  |  19  |  32541  |  11960  |  26130  |  23010  |  True
# 52387  |  2  |  188  |  18538  |  23822  |  43142  |  True
# 46853  |  2  |  37329  |  16241  |  28173  |  288  |  True
# 51907  |  11  |  41378  |  28443  |  15393  |  3906  |  True
# 49043  |  2  |  10860  |  38704  |  47636  |  21866  |  True
# 58657  |  5  |  56047  |  37391  |  54563  |  54660  |  True
# 56359  |  3  |  41128  |  24789  |  4101  |  23727  |  True
# 50503  |  5  |  29593  |  42595  |  24713  |  1901  |  True
# 43801  |  31  |  20495  |  1515  |  19815  |  8895  |  True
# 48527  |  5  |  39012  |  11985  |  31387  |  37068  |  True

p | alpha | beta | r | s | m | test
35543 | 34136 | 16819 | 15288 | 4030 | 5148 | True
37957 | 24525 | 33567 | 19038 | 34950 | 9330 | True
64613 | 46625 | 45445 | 63949 | 9859 | 12447 | True
55843 | 9857 | 22076 | 23168 | 3074 | 17178 | True
55667 | 23501 | 33372 | 26540 | 30066 | 3514 | True
56663 | 51217 | 37397 | 10486 | 46450 | 48348 | True
53617 | 29529 | 18114 | 46739 | 25841 | 34100 | True
55987 | 40325 | 19877 | 50176 | 30800 | 34846 | True
47309 | 12258 | 30005 | 6606 | 38646 | 22760 | True
48383 | 35169 | 34105 | 33931 | 29817 | 39259 | True
